# 1) Goal of the project

The goal of this project is to create the model for heart disease prediction.

It will be a binary classification, where 1 means heart disease and 0 means no disease.

# 2) Data acquisition

The data for this project comes from Kaggle: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

The author of this dataset combined several smaller dataset from the UCI repository, joining them on the common features (attributes).

# 3) Data exploration

The first step in data exploration is loading the data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
PATH = "/content/drive/MyDrive/EWD/Project/Data" # to be changed depending on working environment

In [ ]:
org_df = pd.read_csv(f"{PATH}/heart.csv")
org_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


As we can see, the dataset consists of 918 observations and 12 attributes, in which there is one decision attribute ("HeartDisease").

Also, there are no missing (null) values, as each column consists of 918 non-null entries, which is equal to number of observation.

Data types of all columns are also appropriate.

Next let's analyze each variable one by one.

### Age

In [ ]:
unique_age = org_df["Age"].unique()
unique_age.sort()
print(unique_age)

[28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77]


Border values, i.e. 28 and 77 are reasonable values for age. 

Now let's look at distribution.

In [ ]:
import plotly.figure_factory as ff

In [ ]:
fig = ff.create_distplot([org_df["Age"]], ["Age"], show_rug=False)
fig.update_layout(title="Age Distribution", 
                  yaxis_title="Proportion",
                  xaxis_title="Age",
                  showlegend=False)
fig.show()

We can see Age is roughly normally distributed, although distribution is left-skewed.

### Sex

In [ ]:
org_df["Sex"].value_counts()

M    725
F    193
Name: Sex, dtype: int64

We see there is significantly more males than females in the data. We need to potentially be careful about proper stratification here.

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["Sex"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["Sex"] == "M")]["Sex"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["Sex"] == "F")]["Sex"].size]),
  go.Bar(name="Have disease",
         x=org_df["Sex"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["Sex"] == "M")]["Sex"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["Sex"] == "F")]["Sex"].size]),
])
fig.update_layout(title="Heart disease cases count based on gender",
                  xaxis_title="Gender",
                  yaxis_title="Count")
fig.show()

As we can see, gender variable is potentially important for the model, as it seems from the graph that males might be more vulnerable to heart disease.

### ChestPainType

Legend for values in this column: 

*   TA: Typical Angina
*   ATA: Atypical Angina
*   NAP: Non-Anginal Pain
*   ASY: Asymptomatic

In [ ]:
org_df["ChestPainType"].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

We see that in more than half of patients (observations) did not report any type of chest pain (asymptomatic). Again, we will need to be careful about stratification here, as this variable might be especially significant (even without medical knowledge we can expect, that chest pain might be indication of heart disease).

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["ChestPainType"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] == "ATA")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] == "NAP")]["ChestPainType"].size,
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] == "ASY")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] == "TA")]["ChestPainType"].size]),
  go.Bar(name="Have disease",
         x=org_df["ChestPainType"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] == "ATA")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] == "NAP")]["ChestPainType"].size,
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] == "ASY")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] == "TA")]["ChestPainType"].size]),
])
fig.update_layout(title="Heart disease cases count based on chest pain type",
                  xaxis_title="Chest Pain Type",
                  yaxis_title="Count")
fig.show()

However, by looking at the graph, we might reach opposite conclusion, meaning asymptomatic chest pain might predict heart disease. Let's see at the graph, if we bundle other types of chest pain together.

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=["ASY", "Other"], 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] == "ASY")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ChestPainType"] != "ASY")]["ChestPainType"].size]),
  go.Bar(name="Have disease",
         x=["ASY", "Other"], 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] == "ASY")]["ChestPainType"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ChestPainType"] != "ASY")]["ChestPainType"].size]),
])
fig.update_layout(title="Heart disease cases count based on chest pain type",
                  xaxis_title="Chest Pain Type",
                  yaxis_title="Count")
fig.show()

Above graph is a suggestion, that we might consider making this attribute binary in phase 4 of the project.

### RestingBP

RestingBP: resting blood pressure [mm Hg]

In [ ]:
unique_bp = org_df["RestingBP"].unique()
unique_bp.sort()
print(f"RestingBP values :{unique_bp}")

RestingBP values :[  0  80  92  94  95  96  98 100 101 102 104 105 106 108 110 112 113 114
 115 116 117 118 120 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 148 150 152 154 155 156
 158 160 164 165 170 172 174 178 180 185 190 192 200]


Blood pressure value of 0 is clearly a mistake, which needs to be taken care of (probably this observation should be deleted). Blood pressures above 180 are also suspicious, as such a high blood pressure indicates an emergency situation.

In [ ]:
fig = ff.create_distplot([org_df["RestingBP"]], ["RestingBP"], show_rug=False, bin_size=6)
fig.update_layout(title="RestingBP Distribution", 
                  yaxis_title="Proportion",
                  xaxis_title="RestingBP",
                  showlegend=False)
fig.show()

Distribution is roughly normal, with visible outliers around 0 and 200, that we identified earlier.


### Cholesterol

Cholesterol: serum cholesterol [mg/dl]

In [ ]:
unique_chol = org_df["Cholesterol"].unique()
unique_chol.sort()
print(f"Cholesterol values :{unique_chol}")

Cholesterol values :[  0  85 100 110 113 117 123 126 129 131 132 139 141 142 147 149 152 153
 156 157 159 160 161 163 164 165 166 167 168 169 170 171 172 173 174 175
 176 177 178 179 180 181 182 183 184 185 186 187 188 190 192 193 194 195
 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213
 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231
 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 288 289 290 291 292 293 294 295 297 298 299 300 302 303 304 305
 306 307 308 309 310 311 312 313 315 316 318 319 320 321 322 325 326 327
 328 329 330 331 333 335 336 337 338 339 340 341 342 344 347 349 353 354
 355 358 360 365 369 384 385 388 392 393 394 404 407 409 412 417 458 466
 468 491 518 529 564 603]


In case of cholesterol is is difficult to assess values without specialized knowledge. However, as with blood pressure, I can assume value of 0 is a mistake. I found that optimal level for serum cholesterol is 125–200 mg/dl (https://www.medicalnewstoday.com/articles/321519#optimal-ranges), but I cannot rule out that very high values are possible and are indication of a disease. Also high levels of cholesterol are often related to high blood pressure, so it will be interesting to see if those variables are correlated. 

In [ ]:
fig = ff.create_distplot([org_df["Cholesterol"]], ["Cholesterol"], show_rug=False, bin_size=6)
fig.update_layout(title="Cholesterol Distribution", 
                  yaxis_title="Proportion",
                  xaxis_title="Cholesterol",
                  showlegend=False)
fig.show()

As with RestingBP, we have roughly normal distribution, with substantial proportion of outliers with value 0. Based on distribution, we can also treat value greater than around 450 as outliers, but in our case those observations might actually be relevant.

### FastingBS 

FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]

In [ ]:
org_df["FastingBS"].value_counts()

0    704
1    214
Name: FastingBS, dtype: int64

In [ ]:
org_df["FastingBS"].unique()

array([0, 1])

Similar as with age, we can see that there are significantly more 0's than 1's.

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["FastingBS"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["FastingBS"] == 0)]["FastingBS"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["FastingBS"] == 1)]["FastingBS"].size]),
  go.Bar(name="Have disease",
         x=org_df["FastingBS"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["FastingBS"] == 0)]["FastingBS"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["FastingBS"] == 1)]["FastingBS"].size]),
])
fig.update_layout(title="Heart disease cases count based on fasting blood sugar",
                  xaxis_title="FastingBS",
                  yaxis_title="Count")
fig.show()

Based on the graph above, we might speculate, that having low blood sugar do not predict heart disease, but high blood sugar might predict heart disease.

### RestingECG

RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]

In [ ]:
org_df["RestingECG"].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [ ]:
org_df["RestingECG"].unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

Although I do not know what values other than "Normal" exactly mean in medical terms, they might be potentially important in heart disease prediction, so we should be again be careful about proper stratification. Let's check it on the graph.

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["RestingECG"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["RestingECG"] == "Normal")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["RestingECG"] == "ST")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["RestingECG"] == "LVH")]["RestingECG"].size]),
  go.Bar(name="Have disease",
         x=org_df["RestingECG"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["RestingECG"] == "Normal")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["RestingECG"] == "ST")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["RestingECG"] == "LVH")]["RestingECG"].size]),
])
fig.update_layout(title="Heart disease cases count based on resting electrocardiogram results",
                  xaxis_title="RestingECG",
                  yaxis_title="Count")
fig.show()

It seems this attribute might be the least significant so far. Let's see how graph will look like if we bundle ST nad LVH together.

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=["Normal", "Non-normal"], 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["RestingECG"] == "Normal")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["RestingECG"] != "ST")]["RestingECG"].size]), 
  go.Bar(name="Have disease",
        x=["Normal", "Non-normal"], 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["RestingECG"] == "Normal")]["RestingECG"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["RestingECG"] != "ST")]["RestingECG"].size]), 
])
fig.update_layout(title="Heart disease cases count based on fasting blood sugar",
                  xaxis_title="FastingBS",
                  yaxis_title="Count")
fig.show()

Just by visually examining this graph we can see this attribute might not be relevant and we can exclude it from conisderation to limit the number of dimensions in our dataset.

### MaxHR

MaxHR: maximum heart rate achieved

In [ ]:
unique_hr = org_df["MaxHR"].unique()
unique_hr.sort()
print(f"Max heart rate values :{unique_hr}")

Max heart rate values :[ 60  63  67  69  70  71  72  73  77  78  80  82  83  84  86  87  88  90
  91  92  93  94  95  96  97  98  99 100 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 184 185 186 187 188 190 192 194 195 202]


Values for max heart rates seem to be reasonable, though again, it is difficult to determine without specialized knowledge.

In [ ]:
fig = ff.create_distplot([org_df["MaxHR"]], ["MaxHR"], show_rug=False, bin_size=6)
fig.update_layout(title="MaxHR Distribution", 
                  yaxis_title="Proportion",
                  xaxis_title="MaxHR",
                  showlegend=False)
fig.show()

Although distribution seems roughly normal, it has some slight resemblance to bimodel distribution. On the other hand we do not see any outliers.

### ExerciseAngina

ExerciseAngina: exercise-induced angina [Y: Yes, N: No]

In [ ]:
org_df["ExerciseAngina"].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

Here we have more balanced (closer to uniform) distribution of binary variable.

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["ExerciseAngina"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["ExerciseAngina"] == "N")]["ExerciseAngina"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ExerciseAngina"] == "Y")]["ExerciseAngina"].size]),
  go.Bar(name="Have disease",
         x=org_df["ExerciseAngina"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["ExerciseAngina"] == "N")]["ExerciseAngina"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ExerciseAngina"] == "Y")]["ExerciseAngina"].size]),
])
fig.update_layout(title="Heart disease cases count based on exercise-induced angina",
                  xaxis_title="ExerciseAngina",
                  yaxis_title="Count")
fig.show()

The graph suggests that absence of exercise-induced angina might predict absence of heart disease, while having exercise-induced angina might predict having heart disease.

### Oldpeak

Oldpeak: ST depression induced by exercise relative to rest



I must admit, this attribute is hard for me to interpret, as it refers to some more specialized measurements, so I can only present the distribution of this variable.

In [ ]:
import plotly.express as px

In [ ]:
fig = px.box(org_df, y="Oldpeak")
fig.show()

We can see there are some observations that might be considered outliers, but because of lack of specialized knowledge, I will be careful with delating those observations.

### ST_Slope

ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]

In [ ]:
org_df["ST_Slope"].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

Again, I do not have specialized knowledge to interprest this attribute, so I will just present it graphically. 

In [ ]:
org_df["ST_Slope"].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [ ]:
fig = go.Figure(data=[
  go.Bar(name="No disease",
         x=org_df["ST_Slope"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 0) & (org_df["ST_Slope"] == "Up")]["ST_Slope"].size, 
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ST_Slope"] == "Flat")]["ST_Slope"].size,
            org_df[(org_df["HeartDisease"] == 0) & (org_df["ST_Slope"] == "Down")]["ST_Slope"].size]),
  go.Bar(name="Have disease",
         x=org_df["ST_Slope"].unique(), 
         y=[org_df[(org_df["HeartDisease"] == 1) & (org_df["ST_Slope"] == "Up")]["ST_Slope"].size, 
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ST_Slope"] == "Flat")]["ST_Slope"].size,
            org_df[(org_df["HeartDisease"] == 1) & (org_df["ST_Slope"] == "Down")]["ST_Slope"].size])
])
fig.update_layout(title="Heart disease cases count based on the slope of the peak exercise ST segment",
                  xaxis_title="ST_Slope",
                  yaxis_title="Count")
fig.show()

Even though I cannot provide reasoning behind it, graph suggests this attribute might be relevant for predicting heart disease. Also we might consider combining "Flat" with "Down" and make it binary variable in phase 4 of the project.

### HeartDisease

In [ ]:
org_df["HeartDisease"].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

Finally, our decision attribute, where 1 means having heart disease, and 0 means no heart disease. We can see that there is no big disproportion between 0's and 1's.

### Correlation

Finally, let's consider correlation between variables.

In [ ]:
fig = px.imshow(org_df.corr(), text_auto=True, aspect="auto")
fig.update_layout(title="Correlation matrix")
fig.show()

As we can see, no attributes are strongly correlated. The highest values of correlation are around +/- 0,4. Interestingly, resting blood pressure and level of cholesterol seem completely uncorrelated (value around 0,1), contrary to what we might expect.

# 4) Data transformation

### Data cleaning

First, we will clean the data from observations, that were identified as mistakes (resting blood pressure and cholesterol value of 0).

In [ ]:
clean_df = org_df[(org_df["RestingBP"] > 0) & (org_df["Cholesterol"] > 0)]
print(clean_df.shape)

(746, 12)


In [ ]:
import numpy as np

In [ ]:
print(f"Removed {918 - 746} observations, which is {np.round(((918 - 746) / 918 * 100), 2)}% of original dataset")

Removed 172 observations, which is 18.74% of original dataset


We removed almost 20% of original, which is a big proportion. We might have considered substituting these values with some other value (e.g. mean, median), but that could also be potentially influenced by identified outliers. Another approach would be to remove attributes, but in this project I will go with simple observation removal. Now let's remove attribute identified as irrelevant = RestingECG.

In [ ]:
clean_df.drop("RestingECG", axis=1, inplace=True)

Before performing transformations required for specific model, let's shuffle the data for the purpose of stratification.

In [ ]:
from sklearn.utils import shuffle

In [ ]:
clean_df = shuffle(clean_df, random_state=42)
clean_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
208,28,M,ATA,130,132,0,185,N,0.0,Up,0
259,55,F,ATA,122,320,0,155,N,0.0,Up,0
97,39,M,NAP,160,147,1,160,N,0.0,Up,0
148,50,M,ATA,120,168,0,160,N,0.0,Up,0
567,71,M,ASY,130,221,0,115,Y,0.0,Flat,1


Also, let's seperate decision attribute at that point.

In [ ]:
y_values = clean_df["HeartDisease"]
clean_df.drop("HeartDisease", axis=1, inplace=True)
clean_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
208,28,M,ATA,130,132,0,185,N,0.0,Up
259,55,F,ATA,122,320,0,155,N,0.0,Up
97,39,M,NAP,160,147,1,160,N,0.0,Up
148,50,M,ATA,120,168,0,160,N,0.0,Up
567,71,M,ASY,130,221,0,115,Y,0.0,Flat


I will perform 5-fold cross-validation, and additionally I will leave one more part for final testing, so I will split y_values into 6 parts.

In [ ]:
y_values = np.array_split(y_values, 6)
y_test = y_values.pop(5)

### kNN transformation

Let's start by converting categorical variables into binary variable containing 1 and 0. I realize, that after scaling numerical to range [0,1] those binary variables might be highly influential in the model, as being at the edges of the range, but I accept this limitation in this practice project.

In [ ]:
knn_df = clean_df.copy()
knn_df["Sex"] = [1 if sex == "M" else 0 for sex in knn_df["Sex"]]
knn_df["ChestPainType"] = [1 if pain == "ASY" else 0 for pain in knn_df["ChestPainType"]]
# FastingBS is already binary variable
knn_df["ExerciseAngina"] = [1 if angina == "Y" else 0 for angina in knn_df["ExerciseAngina"]]
knn_df["ST_Slope"] = [1 if slope == "Up" else 0 for slope in knn_df["ST_Slope"]]
knn_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
208,28,1,0,130,132,0,185,0,0.0,1
259,55,0,0,122,320,0,155,0,0.0,1
97,39,1,0,160,147,1,160,0,0.0,1
148,50,1,0,120,168,0,160,0,0.0,1
567,71,1,1,130,221,0,115,1,0.0,0


Now let's scale numerical attributes. I will use only min-max scaling. Sigmoid finction will map almost all value to 1 (or very close to 1), unless we use very small alpha, so I do not think this is a suitable transformation for this particular application.

In [ ]:
attr_to_scale = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
min_max_scaler = MinMaxScaler()
for attr in attr_to_scale:
  knn_df[attr] = min_max_scaler.fit_transform(knn_df[attr].values.reshape(-1, 1))
knn_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
208,0.000000,1,0,0.351852,0.090734,0,0.872180,0,0.015873,1
259,0.551020,0,0,0.277778,0.453668,0,0.646617,0,0.015873,1
97,0.224490,1,0,0.629630,0.119691,1,0.684211,0,0.015873,1
148,0.448980,1,0,0.259259,0.160232,0,0.684211,0,0.015873,1
567,0.877551,1,1,0.351852,0.262548,0,0.345865,1,0.015873,0


### Naive Bayes transformation

In order to stick to traditional implementation of naive Bayes, I will discretize numeric variables. 

In [ ]:
bayes_df = clean_df.copy()

In case of Age, I make arbitrary decision, to split it into 4 bins.

In [ ]:
age_min = bayes_df["Age"].min()
age_max = bayes_df["Age"].max()
age_step = int((age_max - age_min) / 4)
bayes_df["Age"] = pd.cut(x=bayes_df["Age"], 
                         bins=[age_min - 1, age_min + age_step, age_min + 2*age_step, age_min + 3*age_step, 100],
                         labels=[1, 2, 3, 4])

I will try to use some medical knowldege to discretize:



*   RestingBP, based on https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings
*   Cholesterol, based on https://www.healthline.com/health/serum-cholesterol#results
*   MaxHR, based on https://blog.ohiohealth.com/target-heart-rate-exercise/



In [ ]:
bayes_df["RestingBP"] = pd.cut(x=bayes_df["RestingBP"],
                               bins=[0, 119, 129, 139, 179, 300],
                               labels=[1, 2, 3, 4, 5])
bayes_df["Cholesterol"] = [2 if chol >= 200 else 1 for chol in bayes_df["Cholesterol"]]
bayes_df["MaxHR"] = pd.cut(x=bayes_df["MaxHR"],
                           bins=[0, 120, 160, 300],
                           labels=[1, 2, 3])

For Oldpeak, like with Age, I arbitrary split it into 4 bins.

In [ ]:
peak_min = bayes_df["Oldpeak"].min()
peak_max = bayes_df["Oldpeak"].max()
peak_step = (peak_max - peak_min) / 4
bayes_df["Oldpeak"] = pd.cut(x=bayes_df["Oldpeak"], 
                         bins=[peak_min - 1, peak_min + peak_step, peak_min + 2*peak_step, peak_min + 3*peak_step, 100],
                         labels=[1, 2, 3, 4])

Convert other categorical variables into number encoding.

In [ ]:
bayes_df["Sex"] = [2 if sex == "Male" else 1 for sex in bayes_df["Sex"]]
bayes_df["ChestPainType"] = [1 if pain == "ASY" else 2 if pain == "NAP" else 3 if pain == "ATA" else 4 for pain in bayes_df["ChestPainType"]]
bayes_df["ExerciseAngina"] = [2 if angina == "Y" else 1 for angina in bayes_df["ExerciseAngina"]]
bayes_df["ST_Slope"] = [1 if slope == "Up" else 2 if slope == "Flat" else 3 for slope in bayes_df["ST_Slope"]]

In [ ]:
bayes_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
208,1,1,3,3,1,0,3,1,1,1
259,3,1,3,2,2,0,2,1,1,1
97,1,1,2,4,1,1,2,1,1,1
148,2,1,3,2,1,0,2,1,1,1
567,4,1,1,3,2,0,1,2,1,2


### Decision Tree and Random Forest

For sklearn implementation of decision tree, I need to encode categorical variables

In [ ]:
tree_df = clean_df.copy()
tree_df["Sex"] = [2 if sex == "Male" else 1 for sex in tree_df["Sex"]]
tree_df["ChestPainType"] = [1 if pain == "ASY" else 2 if pain == "NAP" else 3 if pain == "ATA" else 4 for pain in tree_df["ChestPainType"]]
tree_df["ExerciseAngina"] = [2 if angina == "Y" else 1 for angina in tree_df["ExerciseAngina"]]
tree_df["ST_Slope"] = [1 if slope == "Up" else 2 if slope == "Flat" else 3 for slope in tree_df["ST_Slope"]]

In [ ]:
tree_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
208,28,1,3,130,132,0,185,1,0.0,1
259,55,1,3,122,320,0,155,1,0.0,1
97,39,1,2,160,147,1,160,1,0.0,1
148,50,1,3,120,168,0,160,1,0.0,1
567,71,1,1,130,221,0,115,2,0.0,2


# 5) Modelling

### kNN

Split dataset into 5 parts for cross-validation, and 1 additional part for final testing.

In [ ]:
knn_df = np.array_split(knn_df, 6)
knn_x_test = knn_df.pop(5)

In kNN I consider 3 different values of k: 3, 5, and 7. As main evaluation metric I choose F-score and I will use this metric to select best hyperparameter. More detailed anaysis of model predictions, includin accuracy, precision and recall, will be done for the final model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [ ]:
F_scores = []
k_params = [3, 5, 7]
# loop through different hyperparameters
for k in k_params:
  F_scores_cross = np.zeros(5)
  # cross-validation loop
  for i in range(5):

    knn_df_copy = knn_df.copy()
    x_eval = knn_df_copy.pop((i+4)%5)
    x_train = pd.concat(knn_df_copy)

    y_values_copy = y_values.copy()
    y_eval = y_values_copy.pop((i+4)%5)
    y_train = pd.concat(y_values_copy)

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)

    pred = knn.predict(x_eval)
    F_scores_cross[i] = f1_score(y_eval, pred)

  F_scores.append(np.mean(F_scores_cross))

for i in range(3):
  print(f"F-score for k = {k_params[i]}: {F_scores[i]}")

F-score for k = 3: 0.8298237385477849
F-score for k = 5: 0.8409486829963223
F-score for k = 7: 0.8415756966898771


The highest score was for k=7, so we will use this parameter for our model.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(pd.concat(knn_df), pd.concat(y_values))

KNeighborsClassifier(n_neighbors=7)

At this point I will only train the final model and evaluate it in phase 6.

### Naive Bayes

For naive Bayes classificator there are no hyperparameters, so I will not perform cross-validation there and go straight to model training. But for the purpose of comparison with outher models, I will also split dataset into 6 parts, using 5 for training and 1 for testing.

In [ ]:
bayes_df = np.array_split(bayes_df, 6)
bayes_x_test = bayes_df.pop(5)

Like with kNN, at this point I only train the model.

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
bayes = GaussianNB()
bayes.fit(pd.concat(bayes_df), pd.concat(y_values))

GaussianNB()

### Decision Tree

In [ ]:
d_tree_df = np.array_split(tree_df, 6)
d_tree_x_test = d_tree_df.pop(5)

For illustration purpose I will generate one decision tree.

In [ ]:
from sklearn import tree

In [ ]:
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(pd.concat(d_tree_df), pd.concat(y_values))

DecisionTreeClassifier()

I considered showing the tree using 
```
tree.plot_tree(decision_tree)
```
but it does not look nice.



### Random Forest

In [ ]:
forest_df = np.array_split(tree_df, 6)
forest_x_test = forest_df.pop(5)

As an extension of decision tree model, I will also use random forest. Here, using sklearn implementation, I will use max depth of a tree as a hyperparameter, that I will fine-tune using cross-validation. I consider (somehow arbitrarily) values from 5 to 10.

As with kNN, I choose f-score as evaluation metric.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
F_scores = []
depths = [5, 6, 7, 8, 9, 10]
# loop through different hyperparameters
for d in depths:
  F_scores_cross = np.zeros(5)
  # cross-validation loop
  for i in range(5):

    tree_df_copy = forest_df.copy()
    x_eval = tree_df_copy.pop((i+4)%5)
    x_train = pd.concat(tree_df_copy)

    y_values_copy = y_values.copy()
    y_eval = y_values_copy.pop((i+4)%5)
    y_train = pd.concat(y_values_copy)

    # for comparison purpose I keep random_state constant
    forest = RandomForestClassifier(max_depth=d, random_state=42)
    forest.fit(x_train, y_train)

    pred = forest.predict(x_eval)
    F_scores_cross[i] = f1_score(y_eval, pred)

  F_scores.append(np.mean(F_scores_cross))

for i in range(6):
  print(f"F-score for max depth = {depths[i]}: {F_scores[i]}")

F-score for max depth = 5: 0.8427662202991952
F-score for max depth = 6: 0.8371427611704634
F-score for max depth = 7: 0.8326113789778207
F-score for max depth = 8: 0.82903994609526
F-score for max depth = 9: 0.8337701595333474
F-score for max depth = 10: 0.8319669320144548


The best f-score is for the smallest value of max depth, so I will experiment a bit more with smaller values.

In [ ]:
F_scores = []
depths = [3, 4, 5]
# loop through different hyperparameters
for d in depths:
  F_scores_cross = np.zeros(5)
  # cross-validation loop
  for i in range(5):

    tree_df_copy = forest_df.copy()
    x_eval = tree_df_copy.pop((i+4)%5)
    x_train = pd.concat(tree_df_copy)

    y_values_copy = y_values.copy()
    y_eval = y_values_copy.pop((i+4)%5)
    y_train = pd.concat(y_values_copy)

    # for comparison purpose I keep random_state constant
    forest = RandomForestClassifier(max_depth=d, random_state=42)
    forest.fit(x_train, y_train)

    pred = forest.predict(x_eval)
    F_scores_cross[i] = f1_score(y_eval, pred)

  F_scores.append(np.mean(F_scores_cross))

for i in range(3):
  print(f"F-score for max depth = {depths[i]}: {F_scores[i]}")

F-score for max depth = 3: 0.8346970449653013
F-score for max depth = 4: 0.8442249753471825
F-score for max depth = 5: 0.8427662202991952


And it turned out that the best hyperparemeter for max depth of the tree is 4 and I will use it for final model training.

In [ ]:
forest = RandomForestClassifier(max_depth=4, random_state=42)
forest.fit(pd.concat(forest_df), pd.concat(y_values))

RandomForestClassifier(max_depth=4, random_state=42)

# 6) Evaluation

### kNN

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
knn_pred = knn.predict(knn_x_test)
print("Confusion matrix:\n")
print(confusion_matrix(y_test, knn_pred))
print("\nClassification report:\n")
print(classification_report(y_test, knn_pred))

Confusion matrix:

[[55 10]
 [10 49]]

Classification report:

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        65
           1       0.83      0.83      0.83        59

    accuracy                           0.84       124
   macro avg       0.84      0.84      0.84       124
weighted avg       0.84      0.84      0.84       124



Looking at confiusion matrix we can see:

*   49 true positives
*   55 true negatives
*   10 false positives
*   10 false negatives

Precision, recall and f-score are all 0.83, while accuracy is slightly higher - 0.84. 

It means out of all of patients (observations) predicted with heart disease 83% indeed had heart disease, and out of all patients with heart disease it was predicted for 83% of them. 



### Naive Bayes

In [ ]:
bayes_pred = bayes.predict(bayes_x_test)
print("Confusion matrix:\n")
print(confusion_matrix(y_test, bayes_pred))
print("\nClassification report:\n")
print(classification_report(y_test, bayes_pred))

Confusion matrix:

[[50 15]
 [ 8 51]]

Classification report:

              precision    recall  f1-score   support

           0       0.86      0.77      0.81        65
           1       0.77      0.86      0.82        59

    accuracy                           0.81       124
   macro avg       0.82      0.82      0.81       124
weighted avg       0.82      0.81      0.81       124



Looking at confiusion matrix we can see:

*   51 true positives
*   50 true negatives
*   15 false positives
*   8 false negatives

Accuracy is 0.81, precision is 0.77, recall is 0.86 and finally f-score is 0.82.

It means out of all of patients (observations) predicted with heart disease 77% indeed had heart disease, and out of all patients with heart disease it was predicted for 86% of them. 

I have chosen f-score as my main evaluation metric, therefore I must conclude kNN is superior in this application, as it has slightly higher f-score. Naive Bayes on the other hand has higher recall and some might argue that this metric is most relevant.

### Decision Tree

In [ ]:
tree_pred = decision_tree.predict(d_tree_x_test)
print("Confusion matrix:\n")
print(confusion_matrix(y_test, tree_pred))
print("\nClassification report:\n")
print(classification_report(y_test, tree_pred))

Confusion matrix:

[[50 15]
 [12 47]]

Classification report:

              precision    recall  f1-score   support

           0       0.81      0.77      0.79        65
           1       0.76      0.80      0.78        59

    accuracy                           0.78       124
   macro avg       0.78      0.78      0.78       124
weighted avg       0.78      0.78      0.78       124



Looking at confiusion matrix we can see:

*   47 true positives
*   50 true negatives
*   15 false positives
*   12 false negatives

Accuracy is 0.78, precision is 0.76, recall is 0.8 and finally f-score is 0.78.

It means out of all of patients (observations) predicted with heart disease 76% indeed had heart disease, and out of all patients with heart disease it was predicted for 80% of them. 

Not surprisingly, single decision tree has f-score lower than both kNN and naive Bayes.

### Random Forest

In [ ]:
forest_pred = forest.predict(forest_x_test)
print("Confusion matrix:\n")
print(confusion_matrix(y_test, forest_pred))
print("\nClassification report:\n")
print(classification_report(y_test, forest_pred))

Confusion matrix:

[[52 13]
 [ 8 51]]

Classification report:

              precision    recall  f1-score   support

           0       0.87      0.80      0.83        65
           1       0.80      0.86      0.83        59

    accuracy                           0.83       124
   macro avg       0.83      0.83      0.83       124
weighted avg       0.83      0.83      0.83       124



Looking at confiusion matrix we can see:

*   51 true positives
*   52 true negatives
*   13 false positives
*   8 false negatives

Accuracy is 0.83, precision is 0.80, recall is 0.86 and finally f-score is 0.83.

It means out of all of patients (observations) predicted with heart disease 80% indeed had heart disease, and out of all patients with heart disease it was predicted for 86% of them. 

Random forest has the same f-score as kNN, so we cannot unambiguously determine, which model is better. But as random forest has higher recall, which might be considered second most important metric, one might consider random forest superior.

# 7) Presentation

On 7th of June in the class.